In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.signal import spectrogram
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical


In [4]:
# Get file list
normal_files = [f for f in os.listdir() if f.endswith('.csv') and 'N' in f]

# Read datasets
dataframes = {f: pd.read_csv(f) for f in normal_files}

# Display available datasets
print("Loaded files:", normal_files)


Loaded files: ['10Nstanding - Sheet1.csv', '11Nstanding - Sheet1.csv', '3Nsitting - Sheet1.csv', '6Nstanding - Sheet1.csv', '5Nstanding - Sheet1.csv', '9Ngait - Sheet1.csv', '8Nstanding - Sheet1.csv', '4Nsitting - Sheet1.csv', '10Ngait - Sheet1.csv', '2Ngait - Sheet1.csv', '3Nstanding - Sheet1.csv', '7Ngait - Sheet1.csv', '5Ngait - Sheet1.csv', '7Nstanding - Sheet1.csv', '1Nstanding - Sheet1.csv', '1Nsitting - Sheet1.csv', '6Nsitting - Sheet1.csv', '1Ngait - 1Nmar.csv', '2Nsitting - Sheet1.csv', '7Nsitting - Sheet1.csv', '4Ngait - Sheet1.csv', '5Nsitting - Sheet1.csv', '8Ngait - Sheet1.csv', '2Nstanding - Sheet1.csv', '3Ngait - Sheet1.csv', '8Nsitting - Sheet1.csv', '4Nstanding - Sheet1.csv', '11Ngait - Sheet1.csv', '9Nstanding - Sheet1.csv', '10Nsitting - Sheet1.csv', '11Nsitting - Sheet1.csv', '6Ngait - Sheet1.csv', '9Nsitting - Sheet1.csv']


In [5]:
# Function to preprocess data
def preprocess_data(df):
    scaler = MinMaxScaler()
    df_scaled = scaler.fit_transform(df.iloc[:, :-1])  # Ignore last column (assumed label)
    return df_scaled

# Function to convert EMG signal into spectrogram
def convert_to_spectrogram(data):
    plt.figure(figsize=(3, 3))
    f, t, Sxx = spectrogram(data, fs=100)  # Assume 100 Hz sampling rate
    plt.pcolormesh(t, f, np.log(Sxx + 1e-10), shading='gouraud')  # Log scale for better contrast
    plt.axis('off')
    plt.close()
    return np.expand_dims(Sxx, axis=-1)  # CNN expects a single channel image

# Process all datasets
processed_data = {name: preprocess_data(df) for name, df in dataframes.items()}


NameError: name 'X_data' is not defined

In [ ]:
# Convert each dataset into image format
X_data, Y_labels = [], []

for name, data in processed_data.items():
    for row in tqdm(data):
        img = convert_to_spectrogram(row)
        X_data.append(img)
        Y_labels.append(0)  # Assuming 0 for normal movement, modify for abnormal data

# Convert to NumPy array
X_data = np.array(X_data)
Y_labels = np.array(Y_labels)

# Train-Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_labels, test_size=0.2, random_state=42)

# One-hot encoding labels
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)
print(f"X_data Shape: {X_data.shape}")
print(f"Y_labels Shape: {Y_labels.shape}")

  0%|                                                 | 0/36900 [00:00<?, ?it/s]/tmp/ipykernel_19338/1893649023.py:10: UserWarning: nperseg = 256 is greater than input length  = 4, using nperseg = 4
  f, t, Sxx = spectrogram(data, fs=100)  # Assume 100 Hz sampling rate
 14%|█████▎                               | 5252/36900 [00:41<03:34, 147.20it/s]